<a href="https://colab.research.google.com/github/sumandutta8877/DATA-Analytics/blob/suman/DA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data from GitHub

In [39]:
!git clone 'https://github.com/sumandutta8877/Data_DA.git'    # Run this Line for only one time

fatal: destination path 'Data_DA' already exists and is not an empty directory.


In [49]:
import pandas as pd
df_train = pd.read_csv('/content/Data_DA/Data/Train_data.csv')
df_test = pd.read_csv('/content/Data_DA/Data/Test_data.csv')
df_sample = pd.read_csv('/content/Data_DA/Data/Sample_submission.csv')
#print(df_train.head)
#print(df_test.head)
#print(df_sample.head)
df1 = df_train            # df1 is the training dataframe
df2 = df_test             # df2 is the test dataframe

# Start Processing

In [50]:
df1['mode'] = df1['mode'].apply(lambda x: 2 if x == 'Major' else 1)
df1['explicit'] = df1['explicit'].apply(lambda x: 0.2 if x == 'No' else 1.2)
df1['id'] = 1
df2['mode'] = df2['mode'].apply(lambda x: 2 if x == 'Major' else 1)
df2['explicit'] = df2['explicit'].apply(lambda x: 0.2 if x == 'No' else 1.2)
df2['id'] = 1
df1.loc[(df1['popularity'] == 'very low'),'ratings'] = 1
df1.loc[(df1['popularity'] == 'low'),'ratings'] = 2
df1.loc[(df1['popularity'] == 'average'),'ratings'] = 3
df1.loc[(df1['popularity'] == 'high'),'ratings'] = 4
df1.loc[(df1['popularity'] == 'very high'),'ratings'] = 5
del df1['popularity']
del df1['year']
del df2['year']
df = df1['id']
df['temp1'] = df1['release_date'].str[0:2].astype(int)
df['temp2'] = df1['release_date'].str[3:5].astype(int)
df['temp3'] = df1['release_date'].str[6:10].astype(int)
df1['loudness'] = -1.0*df1['loudness']
df1['release_date'] = 737300 - (df['temp1']+30.45*df['temp2']+365*df['temp3'])  # number of days old from today
df = df2['id']
df['temp1'] = df2['release_date'].str[0:2].astype(int)
df['temp2'] = df2['release_date'].str[3:5].astype(int)
df['temp3'] = df2['release_date'].str[6:10].astype(int)
df2['loudness'] = -1.0*df2['loudness']
df2['release_date'] = 737300 - (df['temp1']+30.45*df['temp2']+365*df['temp3'])  # number of days old from today

0        27.398
1        10.046
2         8.009
3        15.917
4        12.344
          ...  
12222     5.461
12223    14.014
12224     9.298
12225    26.324
12226     4.765
Name: loudness, Length: 12227, dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [46]:
M_tr = df1.to_numpy()
M_ts = df2.to_numpy()


[[1.      0.949   0.235   ... 0.0398  3.      1.     ]
 [1.      0.855   0.456   ... 0.859   2.4     2.     ]
 [1.      0.827   0.495   ... 0.709   2.6     1.     ]
 ...
 [1.      0.328   0.551   ... 0.89    2.5     2.     ]
 [1.      0.122   0.0608  ... 0.00558 3.1     4.     ]
 [1.      0.038   0.389   ... 0.334   3.1     4.     ]]
